In [1]:
# Устанавливаем зависимости
# !pip install torch transformers seqeval scikit-learn


In [ ]:
USE_NEW_DF = True
TRAIN = True
COMPUTE_METRICS2 = True

trained = False

import ast
import pandas as pd

DATA_URL = "https://gist.githubusercontent.com/Niptlox/77f2302eb799de55081723f63555b524/raw/82522906d4922f1974c339e7f74552a6219b3bb9/train-ner-5.csv"

# Читаем CSV
_df = pd.read_csv(DATA_URL, sep=';')
_df['annotation'].str.replace("'0'", "'O'")
df1=_df

def parse_annotation(s):
    try:
        return ast.literal_eval(s)
    except:
        return []
df1['parsed_annotation'] = df1['annotation'].apply(parse_annotation)
df1['parsed_annotation_n'] = df1['parsed_annotation'].apply(len)

# df2 = pd.read_csv("submission-rbt-(27, 1, 7, 54).csv", sep=';')
# df3 = pd.read_csv("synthetic_submission-rbt-(27, 17, 30, 9).csv", sep=';', nrows=500_000)

print(df1.shape)
df1 = pd.concat([df1] + [df1[df1['annotation'].str.contains("I-TYPE")]] * 3)
print(df1.shape)

(27251, 4)
(40235, 4)


In [3]:
df1

,sample,annotation,parsed_annotation,parsed_annotation_n
0,aa,"[(0, 2, 'O')]","[(0, 2, O)]",1
1,aala,"[(0, 4, 'O')]","[(0, 4, O)]",1
2,aarcca,"[(0, 6, 'O')]","[(0, 6, O)]",1
3,abon,"[(0, 4, 'O')]","[(0, 4, O)]",1
4,abso,"[(0, 4, 'B-BRAND')]","[(0, 4, B-BRAND)]",1
...,...,...,...,...
27135,яйцо с0,"[(0, 4, 'B-TYPE'), (5, 7, 'I-TYPE')]","[(0, 4, B-TYPE), (5, 7, I-TYPE)]",2
27136,яйцо сюрприз,"[(0, 4, 'B-TYPE'), (5, 12, 'I-TYPE')]","[(0, 4, B-TYPE), (5, 12, I-TYPE)]",2
27137,яйцо шоко,"[(0, 4, 'B-TYPE'), (5, 9, 'I-TYPE')]","[(0, 4, B-TYPE), (5, 9, I-TYPE)]",2
27189,японская лапша удон,"[(0, 8, 'B-TYPE'), (9, 14, 'I-TYPE'), (15, 19,...","[(0, 8, B-TYPE), (9, 14, I-TYPE), (15, 19, I-T...",3



```markdown
Привет
```



In [4]:
import random

def add_typo(word: str, lang: str = "ru") -> str:
    """
    Добавляет случайную ошибку в слово.
    Параметры:
    - word: исходное слово (русское или английское)
    - lang: "ru" для русского, "en" для английского (по умолчанию "ru")
    Возвращает слово с одной случайной ошибкой.
    """
    if len(word) < 2:
        return word  # слишком короткое для ошибки

    # Соседние буквы на клавиатуре для русского и английского
    neighbors_ru = {
        'й': 'цш', 'ц': 'йшф', 'у': 'йкх', 'к': 'уеуф', 'е': 'цршг',
        'н': 'гшщз', 'г': 'ерт', 'ш': 'эрту', 'щ': 'шыф', 'з': 'шхэ',
        'х': 'юкэ', 'ф': 'ьыв', 'ы': 'фв', 'в': 'ыпр', 'а': 'пож',
        'п': 'арол', 'р': 'олд', 'о': 'рдл', 'л': 'дяч', 'д': 'ячсм',
        'я': 'чс', 'ч': 'ясм', 'с': 'мти', 'м': 'тиб', 'и': 'цыт',
        'т': 'ишь', 'ь': 'шэ', 'б': 'юь', 'ю': 'бэ'
    }
    neighbors_en = {
        'q': 'w', 'w': 'qes', 'e': 'wrds', 'r': 'etdf', 't': 'rygf',
        'y': 'tugh', 'u': 'yihj', 'i': 'uojk', 'o': 'ipkl', 'p': 'ol',
        'a': 'sqz', 's': 'adwx', 'd': 'serfc', 'f': 'drtg', 'g': 'ftyhb',
        'h': 'gyujn', 'j': 'huikm', 'k': 'jiol', 'l': 'kop', 'z': 'asx',
        'x': 'zsdc', 'c': 'xdfv', 'v': 'cfgb', 'b': 'vghn', 'n': 'bhjm',
        'm': 'njk'
    }

    def get_neighbors(ch):
        ch_lower = ch.lower()
        if lang == 'ru':
            return neighbors_ru.get(ch_lower, '')
        else:
            return neighbors_en.get(ch_lower, '')

    typo_type = random.choice(['delete', 'replace', 'swap'])
    chars = list(word)

    if typo_type == 'delete':
        # Удалить случайный символ (кроме если длина 1)
        pos = random.randint(0, len(chars)-1)
        del chars[pos]

    elif typo_type == 'replace':
        # Заменить символ на соседний на клавиатуре
        pos = random.randint(0, len(chars)-1)
        neighbors = get_neighbors(chars[pos])
        if neighbors:
            replacement = random.choice(neighbors)
            if chars[pos].isupper():
                replacement = replacement.upper()
            chars[pos] = replacement

    elif typo_type == 'insert':
        # Вставить случайный соседний символ возле случайной позиции
        pos = random.randint(0, len(chars)-1)
        neighbors = get_neighbors(chars[pos])
        if neighbors:
            insertion = random.choice(neighbors)
            if chars[pos].isupper():
                insertion = insertion.upper()
            chars.insert(pos, insertion)

    elif typo_type == 'swap':
        # Поменять местами два соседних символа (если длина > 1)
        if len(chars) > 1:
            pos = random.randint(0, len(chars)-2)
            chars[pos], chars[pos+1] = chars[pos+1], chars[pos]

    return ''.join(chars)

# Примеры использования:
print(add_typo("привет", "ru"))
print(add_typo("example", "en"))
df11 = _df[_df['parsed_annotation_n']==1]
df11["sample"] = df11["sample"].apply(add_typo)
df1 = pd.concat([df1, df11])

прывет
exampel


R:\Temp\ipykernel_21520\3810771575.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df11["sample"] = df11["sample"].apply(add_typo)


In [5]:
df1.shape

(55694, 4)

In [ ]:
# Загрузка и подготовка данных
import json
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.model_selection import train_test_split
from seqeval.metrics import f1_score, precision_score, recall_score
from seqeval.scheme import IOB2
import logging
import warnings
warnings.filterwarnings('ignore')

# Логирование
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# URL датасета
DATA_URL = "https://gist.githubusercontent.com/Niptlox/77f2302eb799de55081723f63555b524/raw/82522906d4922f1974c339e7f74552a6219b3bb9/train-ner-5.csv"

# Читаем CSV
if USE_NEW_DF:
    df = df1.copy()
else:
    df = pd.read_csv(DATA_URL, sep=';')
import ast
def parse_annotation(s):
    try:
        return ast.literal_eval(s)
    except:
        return []
df['parsed_annotation'] = df['annotation'].apply(parse_annotation)

# Подготовка данных для NER
tag_to_id = {'O':0,'B-BRAND':1,'I-BRAND':2,'B-TYPE':3,'I-TYPE':4,'B-VOLUME':5,'I-VOLUME':6,'B-PERCENT':7,'I-PERCENT':8}
id_to_tag = {v:k for k,v in tag_to_id.items()}

def prepare_ner_data(df):
    data = []
    for idx, row in df.iterrows():
        sample = row['sample']
        ann = row['parsed_annotation']
        char_tags = ['O']*len(sample)
        for start,end,tag in ann:
            if tag=='0': tag='O'
            for i in range(start, min(end,len(sample))):
                char_tags[i]=tag
        words = sample.split() or list(sample)
        word_tags=[]
        pos=0
        for w in words:
            i=sample.find(w,pos)
            pos=i+len(w)
            word_tags.append(char_tags[i] if i<len(char_tags) else 'O')
        data.append({'words':words,'tags':word_tags})
    return data

ner_data = prepare_ner_data(df)
for ex in ner_data:
    ex['tag_ids'] = [tag_to_id.get(t,0) for t in ex['tags']]


In [18]:
# Датасет и модель
class NERDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data, self.tokenizer, self.max_length = data, tokenizer, max_length
    def __len__(self): return len(self.data)
    def __getitem__(self, idx):
        ex = self.data[idx]
        enc = self.tokenizer(ex['words'], is_split_into_words=True,
                             padding='max_length', truncation=True,
                             max_length=self.max_length, return_tensors='pt')
        labels = []
        word_ids = enc.word_ids()
        prev_idx = None
        for widx in word_ids:
            if widx is None: labels.append(-100)
            elif widx!=prev_idx: labels.append(ex['tag_ids'][widx])
            else: labels.append(-100)
            prev_idx=widx
        return {'input_ids':enc['input_ids'].squeeze(),
                'attention_mask':enc['attention_mask'].squeeze(),
                'labels':torch.tensor(labels)}

class NERModel(nn.Module):
    def __init__(self, model_name, num_tags, dropout=0.3):
        super().__init__()
        self.bert=AutoModel.from_pretrained(model_name)
        self.layer_norm=nn.LayerNorm(self.bert.config.hidden_size)
        self.dropout=nn.Dropout(dropout)
        self.classifier=nn.Linear(self.bert.config.hidden_size, num_tags)
        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)
    def forward(self, input_ids, attention_mask, labels=None, **kwargs):
        out=self.bert(input_ids=input_ids,attention_mask=attention_mask).last_hidden_state
        out=self.layer_norm(out); out=self.dropout(out)
        logits=self.classifier(out)
        loss=None
        if labels is not None:
            loss_fct=nn.CrossEntropyLoss(ignore_index=-100)
            loss=loss_fct(logits.view(-1,logits.shape[-1]), labels.view(-1))
        return {'loss':loss,'logits':logits}


In [19]:
# Вспомогательные функции
def clean_bio(tags):
    cleaned=[]; prev='O'
    for tag in tags:
        if tag.startswith('I-') and not prev.endswith(tag[2:]):
            tag='B-'+tag[2:]
        cleaned.append(tag); prev=tag
    return cleaned

def compute_metrics(preds_labels):
    preds, labels = preds_labels
    preds = np.argmax(preds, axis=2)
    true_preds, true_labels = [], []
    for p,l in zip(preds, labels):
        tp=[id_to_tag[p_i] for p_i, l_i in zip(p,l) if l_i!=-100]
        tl=[id_to_tag[l_i] for p_i, l_i in zip(p,l) if l_i!=-100]
        true_preds.append(clean_bio(tp)); true_labels.append(clean_bio(tl))
    return {
        'precision': precision_score(true_labels, true_preds, mode='strict', scheme=IOB2),
        'recall': recall_score(true_labels, true_preds, mode='strict', scheme=IOB2),
        'f1': f1_score(true_labels, true_preds, mode='strict', scheme=IOB2)
    }


In [20]:
import numpy as np
from collections import defaultdict, Counter
from math import isfinite

# Ожидается наличие: id_to_tag (list/dict), clean_bio (fn)
# Опционально можно настроить порог "неуверенности"
LOW_MARGIN_THRESH = 0.2  # по softmax: средний (p1 - p2) < 0.2

def _tags_to_spans(tags):
    """
    Превращает BIO-последовательность тегов в список спанов:
    возвращает [(start, end_exclusive, type), ...]
    """
    spans = []
    start, cur_type = None, None
    for i, t in enumerate(tags + ["O"]):  # добавим "O" для сброса в конце
        if t == "O" or t.startswith("B-"):
            # закрыть предыдущую сущность
            if cur_type is not None:
                spans.append((start, i, cur_type))
                start, cur_type = None, None
            # открыть новую, если B-
            if t.startswith("B-"):
                start, cur_type = i, t[2:]
        elif t.startswith("I-"):
            # продолжаем, если тот же тип, иначе считаем новой B-
            typ = t[2:]
            if cur_type is None or typ != cur_type:
                # некорректный I-* => старт новой сущности
                if cur_type is not None:
                    spans.append((start, i, cur_type))
                start, cur_type = i, typ
        else:
            # неизвестный тег — сброс
            if cur_type is not None:
                spans.append((start, i, cur_type))
                start, cur_type = None, None
    return spans

def _overlap(a, b):
    # a,b = (s,e,t), полуинтервалы [s,e)
    return not (a[1] <= b[0] or b[1] <= a[0])

def _length_bucket(length):
    if length == 1: return "len_1"
    if 2 <= length <= 3: return "len_2_3"
    return "len_4_plus"




def compute_metrics2(preds_labels):
    preds, labels = preds_labels  # preds: logits (batch, seq, C) или np.array; labels: (batch, seq)
    # Сохраним логиты для "уверенности"
    logits = preds
    # Аргмакс по меткам
    pred_ids = np.argmax(preds, axis=2)

    true_preds, true_labels = [], []
    for p, l in zip(pred_ids, labels):
        tp = [id_to_tag[p_i] for p_i, l_i in zip(p, l) if l_i != -100]
        tl = [id_to_tag[l_i] for p_i, l_i in zip(p, l) if l_i != -100]
        true_preds.append(clean_bio(tp))
        true_labels.append(clean_bio(tl))

    # === Базовые метрики seqeval ===
    from seqeval.metrics import precision_score, recall_score, f1_score, classification_report

    overall_precision = precision_score(true_labels, true_preds, mode='strict', scheme=IOB2)
    overall_recall    = recall_score(true_labels, true_preds, mode='strict', scheme=IOB2)
    overall_f1        = f1_score(true_labels, true_preds, mode='strict', scheme=IOB2)

    # === Диагностика на уровне сущностей ===
    tp_exact = 0
    fp = 0
    fn = 0

    boundary_fp = 0
    boundary_fn = 0
    type_fp = 0
    type_fn = 0
    both_fp = 0
    both_fn = 0

    gold_per_type = Counter()
    pred_per_type = Counter()
    tp_per_type   = Counter()

    # Для длин
    gold_len_cnt = Counter()
    pred_len_cnt = Counter()
    tp_len_pred  = Counter()  # TP по длинам среди предсказаний
    tp_len_gold  = Counter()  # TP по длинам среди истинных

    # Путаница типов
    type_confusions = Counter()

    # «Неуверенность» по логитам (если доступны формы совпадают)
    low_margin_pred = 0
    total_pred_entities = 0
    low_margin_pred_fp = 0

    for b_idx, (pred_tags, gold_tags) in enumerate(zip(true_preds, true_labels)):
        gold_spans = _tags_to_spans(gold_tags)
        pred_spans = _tags_to_spans(pred_tags)

        gold_set = set(gold_spans)
        pred_set = set(pred_spans)

        # Подсчёт по типам и длинам
        for s,e,t in gold_spans:
            gold_per_type[t] += 1
            gold_len_cnt[_length_bucket(e - s)] += 1
        for s,e,t in pred_spans:
            pred_per_type[t] += 1
            pred_len_cnt[_length_bucket(e - s)] += 1

        # TP exact
        exact_matches = gold_set & pred_set
        tp_exact += len(exact_matches)
        for s,e,t in exact_matches:
            tp_per_type[t] += 1
            tp_len = _length_bucket(e - s)
            tp_len_pred[tp_len] += 1
            tp_len_gold[tp_len] += 1

        # Идентификаторы для быстрых проверок
        gold_by_span = defaultdict(list)
        for g in gold_spans:
            gold_by_span[(g[0], g[1])].append(g)
        pred_by_span = defaultdict(list)
        for p in pred_spans:
            pred_by_span[(p[0], p[1])].append(p)

        # Множества для быстрого поиска перекрытий
        gold_only = [g for g in gold_spans if g not in exact_matches]
        pred_only = [p for p in pred_spans if p not in exact_matches]

        # FP детализация
        for p in pred_only:
            total_pred_entities += 1
            p_span = (p[0], p[1])
            # type-ошибка: есть gold с теми же границами, но другим типом
            same_span_gold = [g for g in gold_by_span[p_span] if g[2] != p[2]]
            if same_span_gold:
                type_fp += 1
                for g in same_span_gold:
                    type_confusions[(g[2], p[2])] += 1
                continue
            # boundary / both / спуриюс
            overlaps = [g for g in gold_only if _overlap(g, p)]
            if overlaps:
                same_type = any(g[2] == p[2] for g in overlaps)
                diff_type = any(g[2] != p[2] for g in overlaps)
                if same_type and not diff_type:
                    boundary_fp += 1
                elif diff_type and not same_type:
                    # чисто типовая на перекрытии (без совпадения границ)
                    type_fp += 1
                    for g in overlaps:
                        if g[2] != p[2]:
                            type_confusions[(g[2], p[2])] += 1
                else:
                    both_fp += 1
                    for g in overlaps:
                        if g[2] != p[2]:
                            type_confusions[(g[2], p[2])] += 1
            else:
                fp += 1  # спуриюс: нет перекрытий вовсе

        # FN детализация
        for g in gold_only:
            # type-ошибка: есть пред с теми же границами, но иным типом
            g_span = (g[0], g[1])
            same_span_pred = [p for p in pred_by_span[g_span] if p[2] != g[2]]
            if same_span_pred:
                type_fn += 1
                for p in same_span_pred:
                    type_confusions[(g[2], p[2])] += 1
                continue
            overlaps = [p for p in pred_only if _overlap(g, p)]
            if overlaps:
                same_type = any(p[2] == g[2] for p in overlaps)
                diff_type = any(p[2] != g[2] for p in overlaps)
                if same_type and not diff_type:
                    boundary_fn += 1
                elif diff_type and not same_type:
                    type_fn += 1
                    for p in overlaps:
                        if p[2] != g[2]:
                            type_confusions[(g[2], p[2])] += 1
                else:
                    both_fn += 1
                    for p in overlaps:
                        if p[2] != g[2]:
                            type_confusions[(g[2], p[2])] += 1
            else:
                fn += 1

        # Подсчёт «неуверенных» предсказанных сущностей по логитам, если доступно
        # Проверим формы
        if isinstance(logits, np.ndarray) and logits.ndim == 3 and b_idx < logits.shape[0]:
            # восстановим логиты только на не-маскированных позициях
            # labels[b_idx] имеет форму последовательности с -100 для игнорируемых
            seq_logits = logits[b_idx]
            seq_labels = labels[b_idx]
            # отфильтруем до длины pred_tags (после удаления -100)
            # соберем logits только на позициях, где label != -100
            keep_idx = [i for i, l_i in enumerate(seq_labels) if l_i != -100]
            if keep_idx:
                kept_logits = np.stack([seq_logits[i] for i in keep_idx], axis=0)  # (T, C)
                # построим карту индексов токенов → в какие сущности входит
                # восстановим токенные индексы сущностей из пред_tags
                # здесь pred_tags уже без -100, индексы сущностей совпадают с позицией в pred_tags
                # пройдёмся по спанам и оценим margin
                for p in pred_only + list(exact_matches):
                    s, e, _ = p
                    # s,e — индексы в последовательности после удаления -100
                    if 0 <= s < e <= kept_logits.shape[0]:
                        # softmax по классу
                        logit_slice = kept_logits[s:e]  # (L, C)
                        # стабильный softmax
                        m = logit_slice.max(axis=1, keepdims=True)
                        probs = np.exp(logit_slice - m) / np.exp(logit_slice - m).sum(axis=1, keepdims=True)
                        # margin top1 - top2
                        part = np.partition(probs, -2, axis=1)
                        top2 = part[:, -2]
                        top1 = part[:, -1]
                        margin = (top1 - top2).mean()
                        total_pred_entities += 0  # уже учтён выше для pred_only; для exact учтём здесь
                        if p in exact_matches:
                            total_pred_entities += 1
                        if not isfinite(margin):
                            continue
                        if margin < LOW_MARGIN_THRESH:
                            # если это FP — учтём отдельно
                            if p in pred_only:
                                low_margin_pred_fp += 1
                            low_margin_pred += 1

    # Подсчёт пер‑классных метрик по сущностям
    per_label = {}
    for t in sorted(set(list(gold_per_type.keys()) + list(pred_per_type.keys()))):
        tp_t = tp_per_type[t]
        p_t  = pred_per_type[t]
        g_t  = gold_per_type[t]
        prec = tp_t / p_t if p_t > 0 else 0.0
        rec  = tp_t / g_t if g_t > 0 else 0.0
        f1   = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
        per_label[t] = (prec, rec, f1, g_t)

    # Найдём «худшие» классы по F1 при достаточной поддержке
    MIN_SUPPORT = 20
    worst = sorted([(t, v[2], v[3]) for t, v in per_label.items() if v[3] >= MIN_SUPPORT], key=lambda x: x[1])[:3]
    worst_labels = [f"{t}: f1={f1:.3f}, support={sup}" for t, f1, sup in worst]

    # F1 по длинам (используем раздельные precision/recall по «пред» и «голд» корзинам)
    def _bucket_f1(tp_pred_cnt, pred_cnt, tp_gold_cnt, gold_cnt, name):
        prec = tp_pred_cnt[name] / pred_cnt[name] if pred_cnt[name] > 0 else 0.0
        rec  = tp_gold_cnt[name] / gold_cnt[name] if gold_cnt[name] > 0 else 0.0
        f1   = 2 * prec * rec / (prec + rec) if (prec + rec) > 0 else 0.0
        return f1, prec, rec

    f1_l1, p_l1, r_l1 = _bucket_f1(tp_len_pred, pred_len_cnt, tp_len_gold, gold_len_cnt, "len_1")
    f1_l23, p_l23, r_l23 = _bucket_f1(tp_len_pred, pred_len_cnt, tp_len_gold, gold_len_cnt, "len_2_3")
    f1_l4p, p_l4p, r_l4p = _bucket_f1(tp_len_pred, pred_len_cnt, tp_len_gold, gold_len_cnt, "len_4_plus")

    # Интегральные индикаторы ошибок
    total_gold_entities = sum(gold_per_type.values())
    total_pred_entities_logged = sum(pred_per_type.values())

    boundary_err_rate = (boundary_fn + boundary_fp) / max(1, total_gold_entities + total_pred_entities_logged)
    type_err_rate     = (type_fn + type_fp) / max(1, total_gold_entities + total_pred_entities_logged)
    both_err_rate     = (both_fn + both_fp) / max(1, total_gold_entities + total_pred_entities_logged)

    # Топ путаниц типов
    top_confusions = type_confusions.most_common(5)
    top_confusions_str = "; ".join([f"{a}->{b}: {c}" for (a,b), c in top_confusions]) if top_confusions else ""

    # Доля «неуверенных» среди предсказанных сущностей
    total_preds_for_margin = total_pred_entities if total_pred_entities > 0 else 1
    low_margin_rate_pred = low_margin_pred / total_preds_for_margin
    low_margin_rate_pred_fp = low_margin_pred_fp / max(1, (len(pred_only) if 'pred_only' in locals() else 0))

    return {
        # базовые
        'precision': overall_precision,
        'recall': overall_recall,
        'f1': overall_f1,

        # длины спанов
        'f1_len1': f1_l1, 'p_len1': p_l1, 'r_len1': r_l1,
        'f1_len2_3': f1_l23, 'p_len2_3': p_l23, 'r_len2_3': r_l23,
        'f1_len4_plus': f1_l4p, 'p_len4_plus': p_l4p, 'r_len4_plus': r_l4p,

        # типы ошибок
        'boundary_err_rate': boundary_err_rate,
        'type_err_rate': type_err_rate,
        'both_err_rate': both_err_rate,

        # редкие/сложные классы
        'num_gold_entities': total_gold_entities,
        'num_pred_entities': total_pred_entities_logged,
        'worst_labels': "; ".join(worst_labels),

        # путаницы типов
        'type_confusions_top5': top_confusions_str,

        # уверенность модели
        'low_margin_rate_pred': low_margin_rate_pred,
        'low_margin_rate_pred_fp': low_margin_rate_pred_fp,
    }


In [21]:
# Основная функция обучения
def train_model():
    MODEL="cointegrated/rubert-tiny2"; MAX_LEN=32; BS=32; LR=3e-5; EPOCHS=10; WARM=500
    data=ner_data
    # Очищаем BIO
    for ex in data:
        ex['tags']=clean_bio(ex['tags'])
        ex['tag_ids']=[tag_to_id[t] for t in ex['tags']]
    train, val = train_test_split(data, test_size=0.15, random_state=42,
                                  stratify=[0.4 if any(t!='O' for t in ex['tags']) else 0 for ex in data])
    tokenizer=AutoTokenizer.from_pretrained(MODEL)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model=NERModel(MODEL, len(tag_to_id)).to(device)
    train_ds, val_ds = NERDataset(train, tokenizer, MAX_LEN), NERDataset(val, tokenizer, MAX_LEN)
    args=TrainingArguments(
        output_dir='./ner_out', num_train_epochs=EPOCHS, per_device_train_batch_size=BS,
        per_device_eval_batch_size=BS, learning_rate=LR, weight_decay=0.01,
        logging_dir='./logs', logging_steps=200, eval_strategy="steps", eval_steps=500,
        save_strategy="steps", save_steps=500, load_best_model_at_end=True,
        metric_for_best_model="f1", greater_is_better=True, warmup_steps=WARM,
        gradient_accumulation_steps=2, fp16=True, remove_unused_columns=False)
    trainer=Trainer(model=model, args=args, train_dataset=train_ds, eval_dataset=val_ds,
                    tokenizer=tokenizer, compute_metrics=compute_metrics2 if COMPUTE_METRICS2 else compute_metrics,
                    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)])
    logger.info("Starting training")
    trainer.train()
    eval_res=trainer.evaluate()
    logger.info(f"Eval results: {eval_res}")
    trainer.save_model("./best_ner_model")
    tokenizer.save_pretrained("./best_ner_model")
    with open("./best_ner_model/config.json","w",encoding="utf-8") as f:
        json.dump({'tag_to_id':tag_to_id,'id_to_tag':id_to_tag,'model':MODEL,'max_len':MAX_LEN},f,ensure_ascii=False,indent=2)
    return trainer, eval_res
if TRAIN:
    trained = True
    # Запуск обучения
    trainer, results = train_model()
    print("\n" + "="*50)
    print("Training complete!")
    print(f"F1-score: {results.get('eval_f1'):.4f}")
    print("="*50)

INFO:__main__:Starting training


Step,Training Loss,Validation Loss,Precision,Recall,F1,F1 Len1,P Len1,R Len1,F1 Len2 3,P Len2 3,R Len2 3,F1 Len4 Plus,P Len4 Plus,R Len4 Plus,Boundary Err Rate,Type Err Rate,Both Err Rate,Num Gold Entities,Num Pred Entities,Worst Labels,Type Confusions Top5,Low Margin Rate Pred,Low Margin Rate Pred Fp
500,1.181200,0.384981,0.876447,0.863536,0.869944,0.864944,0.892055,0.839432,0.882741,0.846338,0.922417,0.142857,0.142857,0.142857,0.037550,0.057412,0.019974,9028,8895,"BRAND: f1=0.648, support=1377; TYPE: f1=0.904, support=7625",BRAND->TYPE: 1200; TYPE->BRAND: 164; PERCENT->TYPE: 10; VOLUME->BRAND: 10; VOLUME->TYPE: 6,0.012029,75.000000
1000,0.653100,0.295521,0.896438,0.906070,0.901228,0.891044,0.890763,0.891325,0.926347,0.910335,0.942932,0.181818,0.250000,0.142857,0.024238,0.049083,0.011513,9028,9125,"BRAND: f1=0.731, support=1377; TYPE: f1=0.930, support=7625",BRAND->TYPE: 898; TYPE->BRAND: 170; VOLUME->BRAND: 8; PERCENT->TYPE: 8; PERCENT->VOLUME: 6,0.015233,82.000000
1500,0.553100,0.228483,0.913847,0.930549,0.922123,0.914049,0.905698,0.922555,0.942546,0.933748,0.951511,0.200000,0.333333,0.142857,0.019209,0.038472,0.007683,9028,9193,"BRAND: f1=0.798, support=1377; TYPE: f1=0.944, support=7625",BRAND->TYPE: 642; TYPE->BRAND: 176; PERCENT->VOLUME: 6; PERCENT->BRAND: 4; VOLUME->BRAND: 4,0.016426,75.000000
2000,0.457200,0.195830,0.925095,0.941183,0.933070,0.924073,0.914440,0.933912,0.955473,0.950535,0.960463,0.250000,1.000000,0.142857,0.015429,0.033987,0.005820,9028,9185,"BRAND: f1=0.825, support=1377; TYPE: f1=0.952, support=7625",BRAND->TYPE: 546; TYPE->BRAND: 162; PERCENT->VOLUME: 6; VOLUME->PERCENT: 6; PERCENT->TYPE: 4,0.013609,67.000000
2500,0.390200,0.189042,0.930929,0.940518,0.935699,0.927825,0.923979,0.931703,0.955251,0.947195,0.963446,0.250000,1.000000,0.142857,0.015263,0.032068,0.005234,9028,9121,"BRAND: f1=0.832, support=1377; TYPE: f1=0.954, support=7625",BRAND->TYPE: 540; TYPE->BRAND: 120; PERCENT->VOLUME: 6; VOLUME->PERCENT: 6; PERCENT->TYPE: 4,0.011622,63.000000
3000,0.362200,0.178239,0.936513,0.952592,0.944484,0.935081,0.925263,0.945110,0.967694,0.963401,0.972025,0.444444,1.000000,0.285714,0.011861,0.029213,0.003020,9028,9183,"BRAND: f1=0.858, support=1377; TYPE: f1=0.960, support=7625",BRAND->TYPE: 468; TYPE->BRAND: 112; PERCENT->VOLUME: 6; VOLUME->PERCENT: 2; BRAND->VOLUME: 2,0.009910,44.000000
3500,0.310600,0.178745,0.938902,0.954918,0.946842,0.938048,0.930478,0.945741,0.968040,0.959004,0.977247,0.769231,0.833333,0.714286,0.010983,0.027457,0.003295,9028,9182,"BRAND: f1=0.863, support=1377; TYPE: f1=0.962, support=7625",BRAND->TYPE: 420; TYPE->BRAND: 128; PERCENT->TYPE: 4; VOLUME->PERCENT: 4; PERCENT->VOLUME: 4,0.007515,40.000000
4000,0.291100,0.161877,0.942998,0.954696,0.948811,0.938964,0.932897,0.945110,0.972562,0.966826,0.978366,0.727273,1.000000,0.571429,0.009742,0.026750,0.002862,9028,9140,"BRAND: f1=0.865, support=1377; TYPE: f1=0.964, support=7625",BRAND->TYPE: 390; TYPE->BRAND: 140; PERCENT->VOLUME: 4; VOLUME->PERCENT: 2; BRAND->VOLUME: 2,0.010503,54.000000
4500,0.280600,0.165592,0.943470,0.959459,0.951398,0.942187,0.933437,0.951104,0.973684,0.967587,0.979858,0.769231,0.833333,0.714286,0.009611,0.025592,0.002416,9028,9181,"BRAND: f1=0.872, support=1377; TYPE: f1=0.965, support=7625",BRAND->TYPE: 390; TYPE->BRAND: 116; VOLUME->PERCENT: 4; PERCENT->TYPE: 2,0.008714,39.000000
5000,0.249600,0.163180,0.943772,0.959349,0.951497,0.941480,0.932807,0.950315,0.975709,0.970133,0.981350,0.769231,0.833333,0.714286,0.009118,0.025872,0.002362,9028,9177,"BRAND: f1=0.872, support=1377; TYPE: f1=0.966, support=7625",BRAND->TYPE: 406; TYPE->BRAND: 102; PERCENT->VOLUME: 4; VOLUME->PERCENT: 2; PERCENT->TYPE: 2,0.009153,50.000000


INFO:__main__:Eval results: {'eval_loss': 0.15740543603897095, 'eval_precision': 0.946691376866892, 'eval_recall': 0.9618963225520603, 'eval_f1': 0.9542332838855007, 'eval_f1_len1': 0.9440799750078102, 'eval_p_len1': 0.9350247524752475, 'eval_r_len1': 0.9533123028391167, 'eval_f1_len2_3': 0.9788261515601783, 'eval_p_len2_3': 0.9748427672955975, 'eval_r_len2_3': 0.9828422230511004, 'eval_f1_len4_plus': 0.7692307692307692, 'eval_p_len4_plus': 0.8333333333333334, 'eval_r_len4_plus': 0.7142857142857143, 'eval_boundary_err_rate': 0.00802153727817153, 'eval_type_err_rate': 0.025108510521399924, 'eval_both_err_rate': 0.0018680292291632329, 'eval_num_gold_entities': 9028, 'eval_num_pred_entities': 9173, 'eval_worst_labels': 'BRAND: f1=0.876, support=1377; TYPE: f1=0.968, support=7625', 'eval_type_confusions_top5': 'BRAND->TYPE: 354; TYPE->BRAND: 134; VOLUME->PERCENT: 2; PERCENT->TYPE: 2', 'eval_low_margin_rate_pred': 0.0094843562629456, 'eval_low_margin_rate_pred_fp': 45.0, 'eval_runtime': 4.8


Training complete!
F1-score: 0.9542


In [22]:
import torch
from tqdm import tqdm

import torch
import numpy as np
from tqdm import tqdm

def _word_level_decode_from_logits(logits, tokenizer):
    # logits: (Lsubtok, C); вернуть список предсказанных id на уровне слов
    pred_ids = logits.argmax(axis=-1)
    # word_ids для первого (единственного) элемента батча
    # Важно: использовать fast токенизатор с .word_ids
    word_ids = tokenizer.word_ids(0) if hasattr(tokenizer, "word_ids") else tokenizer.encodings[0].word_ids
    word_pred_ids = []
    seen = set()
    for t_idx, w_idx in enumerate(word_ids):
        if w_idx is None:
            continue
        if w_idx not in seen:
            word_pred_ids.append(int(pred_ids[t_idx]))
            seen.add(w_idx)
    return word_pred_ids

import torch, numpy as np
from tqdm import tqdm

def make_id2tag_mapper(id_to_tag, fallback_id2label=None):
    # id_to_tag может быть dict или function; fallback_id2label — dict из config
    if callable(id_to_tag):
        return lambda i: id_to_tag(int(i))
    if isinstance(id_to_tag, dict):
        def _map(i):
            i = int(i)
            # поддержка dict с int или str ключами
            return id_to_tag.get(i, id_to_tag.get(str(i), "O"))
        return _map
    if isinstance(fallback_id2label, dict):
        return lambda i: fallback_id2label.get(int(i), fallback_id2label.get(str(int(i)), "O"))
    raise TypeError("id_to_tag должен быть dict или callable")

def word_level_pred_ids(logits_1ex, wids):
    # logits_1ex: (Lsubtok, C)
    pred_ids = logits_1ex.argmax(axis=-1)
    # word_ids из encoding (fast токенизатор)
    res, seen = [], set()
    for t_idx, w_idx in enumerate(wids):
        if w_idx is None:
            continue
        if w_idx not in seen:
            res.append(int(pred_ids[t_idx]))
            seen.add(w_idx)
    return res

def test_model_errors(trainer, val_dataset, tokenizer, id_to_tag,
                      tag_pad_id=-100, max_length=128, device=None, max_examples_print=20):
    model = trainer.model
    model.eval()
    if device is None:
        device = next(model.parameters()).device

    id2label_cfg = getattr(getattr(trainer, "model", None), "config", None)
    id2label_cfg = getattr(id2label_cfg, "id2label", None)
    label2id_cfg = getattr(getattr(trainer, "model", None), "config", None)
    label2id_cfg = getattr(label2id_cfg, "label2id", None)

    map_tag = make_id2tag_mapper(id_to_tag, fallback_id2label=id2label_cfg)

    errors, shown = [], 0
    n = len(val_dataset)

    for i in tqdm(range(n), desc="Testing model"):
        ex = val_dataset[i]
        words = ex.get("words")
        if words is None:
            raise KeyError("В элементе вал. набора нет 'words' — адаптируйте доступ к данным.")  [attached_file:1]

        # Берём id-теги, либо восстанавливаем из строковых тегов
        gold_ids = ex.get("tag_ids") or ex.get("labels") or ex.get("ner_tags")
        if gold_ids is None:
            tags_str = ex.get("tags")
            if tags_str is None:
                raise KeyError("Нет 'tag_ids'/'labels'/'ner_tags' и 'tags' для восстановления.")  [attached_file:1]
            if not isinstance(label2id_cfg, dict):
                raise KeyError("Невозможно восстановить tag_ids: отсутствует label2id в config.")  [attached_file:1]
            def to_id(t):
                # поддержка label2id с ключами str
                return int(label2id_cfg.get(t, label2id_cfg.get(str(t), 0)))
            gold_ids = [to_id(t) for t in tags_str]

        # Токенизация на лету
        enc = tokenizer(
            words,
            is_split_into_words=True,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )
        wids = enc.word_ids(batch_index=0)  # fast tokenizer API

        enc = {k: v.to(device) for k, v in enc.items()}
        with torch.no_grad():
            out = model(**enc)
            logits = out["logits"][0].detach().cpu().numpy()  # (Lsubtok, C)

        # Предсказания на уровне слов
        pred_word_ids = word_level_pred_ids(logits, wids)
        pred_tags = [map_tag(pid) for pid in pred_word_ids]

        # GOLD теги на уровне слов
        # Предполагается, что gold_ids соответствуют словам по индексам
        gold_tags = [map_tag(t) for t in gold_ids if int(t) != tag_pad_id]

        # Уравниваем длины
        L = min(len(words), len(gold_tags), len(pred_tags))
        words_s = words[:L]
        gold_s = gold_tags[:L]
        pred_s = pred_tags[:L]

        token_diffs = [(k, words_s[k], gold_s[k], pred_s[k]) for k in range(L) if gold_s[k] != pred_s[k]]
        if token_diffs:
            if shown < max_examples_print:
                print("="*80)
                print(f"idx={i}")
                print("TEXT:", " ".join(words_s))
                print("GOLD:", " ".join(gold_s))
                print("PRED:", " ".join(pred_s))
                print("Token diffs:")
                for pos, tok, gt, pt in token_diffs:
                    print(f"  [{pos}] {tok} :: {gt} -> {pt}")
                shown += 1

            errors.append({
                "idx": i,
                "text": " ".join(words_s),
                "words": words_s,
                "gold": gold_s,
                "pred": pred_s,
                "token_diffs": token_diffs,
            })

    print("="*80)
    print(f"Found {len(errors)} errors in the validation set")
    return errors


# Запуск тестировщика
if trained:
    logger.info("Testing model on validation set...")
    errors = test_model_errors(trainer, ner_data, trainer.tokenizer, id_to_tag)

    # Вывод ошибок
    print(f"\n{'='*50}\nFound {len(errors)} errors in the validation set:\n{'='*50}")
    for i, error in enumerate(errors[:10], 1):  # Выводим первые 10 ошибок
        print(f"\nError #{i}:")
        print(f"Text: {error['text']}")
        # print(f"Tokens: {' '.join(error['tokens'])}")
        print(f"True tags: {' '.join(error['gold'])}")
        print(f"Pred tags: {' '.join(error['pred'])}")
        print("Differences:")
        for idx, word, true_tag, pred_tag in error['token_diffs']:
            print(f"  Word '{word}': True={true_tag}, Pred={pred_tag}")
    with open("test_model_errors.json", "w") as f:
        json.dump(errors, f)

INFO:__main__:Testing model on validation set...
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


Testing model:   0%|          | 14/55694 [00:00<06:50, 135.79it/s]

idx=3
TEXT: abon
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] abon :: O -> B-BRAND
idx=17
TEXT: active
GOLD: B-BRAND
PRED: O
Token diffs:
  [0] active :: B-BRAND -> O


Testing model:   0%|          | 90/55694 [00:00<06:08, 150.95it/s]

idx=67
TEXT: alpin gold he
GOLD: B-BRAND I-BRAND O
PRED: B-BRAND I-BRAND I-BRAND
Token diffs:
  [2] he :: O -> I-BRAND
idx=79
TEXT: aos
GOLD: B-BRAND
PRED: O
Token diffs:
  [0] aos :: B-BRAND -> O
idx=80
TEXT: aotime
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] aotime :: O -> B-BRAND
idx=92
TEXT: apy ople
GOLD: O O
PRED: B-BRAND O
Token diffs:
  [0] apy :: O -> B-BRAND
idx=97
TEXT: aqva
GOLD: B-TYPE
PRED: B-BRAND
Token diffs:
  [0] aqva :: B-TYPE -> B-BRAND


...

idx=258
TEXT: bic
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] bic :: O -> B-BRAND
idx=279
TEXT: bled
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] bled :: O -> B-BRAND
idx=282
TEXT: blue
GOLD: B-BRAND
PRED: B-TYPE
Token diffs:
  [0] blue :: B-BRAND -> B-TYPE
idx=287
TEXT: bodyr
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] bodyr :: O -> B-BRAND
idx=290
TEXT: boge
GOLD: O
PRED: B-BRAND
Token diffs:
  [0] boge :: O -> B-BRAND


Testing model:   1%|          | 338/55694 [00:02<05:34, 165.44it/s]

idx=311
TEXT: bonelama
GOLD: O
PRED: B-TYPE
Token diffs:
  [0] bonelama :: O -> B-TYPE


Testing model: 100%|██████████| 55694/55694 [05:30<00:00, 168.49it/s]

Found 2187 errors in the validation set

Found 2187 errors in the validation set:

Error #1:
Text: abon
True tags: O
Pred tags: B-BRAND
Differences:
  Word 'abon': True=O, Pred=B-BRAND

Error #2:
Text: active
True tags: B-BRAND
Pred tags: O
Differences:
  Word 'active': True=B-BRAND, Pred=O

Error #3:
Text: alpin gold he
True tags: B-BRAND I-BRAND O
Pred tags: B-BRAND I-BRAND I-BRAND
Differences:
  Word 'he': True=O, Pred=I-BRAND

Error #4:
Text: aos
True tags: B-BRAND
Pred tags: O
Differences:
  Word 'aos': True=B-BRAND, Pred=O

Error #5:
Text: aotime
True tags: O
Pred tags: B-BRAND
Differences:
  Word 'aotime': True=O, Pred=B-BRAND

Error #6:
Text: apy ople
True tags: O O
Pred tags: B-BRAND O
Differences:
  Word 'apy': True=O, Pred=B-BRAND

Error #7:
Text: aqva
True tags: B-TYPE
Pred tags: B-BRAND
Differences:
  Word 'aqva': True=B-TYPE, Pred=B-BRAND

Error #8:
Text: aroma
True tags: B-TYPE
Pred tags: B-BRAND
Differences:
  Word 'aroma': True=B-TYPE, Pred=B-BRAND

Error #9:
Text: asi

In [23]:
# Ячейка для корректной загрузки модели из best_ner_model с указанием конфигурации

from transformers import AutoTokenizer, AutoConfig, AutoModelForTokenClassification
import torch

# Загружаем словарь тегов из сохранённого config.json
import json

model_path = "./best_ner_model"
try:
    if not trained:
        model_path = r".\submission\0.9162\best_ner_model"
except:
    pass

with open(model_path+"/config.json", "r", encoding="utf-8") as f:
    saved = json.load(f)
tag_to_id = saved["tag_to_id"]
id_to_tag = saved["id_to_tag"]
num_labels = len(tag_to_id)

# Загружаем базовую конфигурацию предобученной модели и обновляем для задачи NER
base_model_name = saved.get("model", "cointegrated/rubert-tiny2")
config = AutoConfig.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    id2label=id_to_tag,
    label2id=tag_to_id
)

# Загружаем токенизатор и модель весов
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForTokenClassification.from_pretrained(
    model_path,
    config=config,
    trust_remote_code=False
)

# Переносим на устройство и в режим оценки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-2): 3 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12,

In [24]:
def predict(text: str) -> str:
    words = text.split()
    enc = tokenizer(
        words,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=saved.get("max_len", 128),
        return_tensors="pt"
    ).to(device)

    with torch.no_grad():
        logits = model(**enc).logits

    preds = logits.argmax(dim=-1)[0].cpu().tolist()
    word_ids = enc.word_ids(batch_index=0)

    result_tags = []
    result = []
    prev_widx = None
    for idx, widx in enumerate(word_ids):
        if widx is not None and widx != prev_widx:
            tag = id_to_tag[str(preds[idx])] if isinstance(list(id_to_tag.keys())[0], str) else id_to_tag[preds[idx]]
            token = words[widx]
            result.append(f"{token}[{tag}]")
            prev_widx = widx
            result_tags.append(tag)

    return " ".join(result), result_tags

samples_to_clear = [   "молоко простоквашино 3.2",
    "хлеб бородинский черкизово",
    "кока кола 2 литра",
    "масло сливочное 82%",
    "йогурт данон клубника"
    "форма для выпечки", "фарш свиной", "сок ананасовый без сахара", "еринги",
    "молооко", "поролок", "свмнина", "припрааа для рыбы", "сулугцн",
    "mitti", "пастеризованна", "полуфабрикаиы", "семена экон",
    "соус по грузински", "то матная паста", "фаворит",
    "ополаскиватель. для белья", "сливкп", "бамбу", "nishoom", "семушка орех"
]
# Тестирование
for sample in samples_to_clear:
    print(f"Input: {sample}\nOutput: {predict(sample)}\n")

Input: молоко простоквашино 3.2
Output: ('молоко[B-TYPE] простоквашино[B-BRAND] 3.2[B-VOLUME]', ['B-TYPE', 'B-BRAND', 'B-VOLUME'])

Input: хлеб бородинский черкизово
Output: ('хлеб[B-TYPE] бородинский[I-TYPE] черкизово[B-BRAND]', ['B-TYPE', 'I-TYPE', 'B-BRAND'])

Input: кока кола 2 литра
Output: ('кока[B-TYPE] кола[I-BRAND] 2[B-VOLUME] литра[I-VOLUME]', ['B-TYPE', 'I-BRAND', 'B-VOLUME', 'I-VOLUME'])

Input: масло сливочное 82%
Output: ('масло[B-TYPE] сливочное[I-TYPE] 82%[B-PERCENT]', ['B-TYPE', 'I-TYPE', 'B-PERCENT'])

Input: йогурт данон клубникаформа для выпечки
Output: ('йогурт[B-TYPE] данон[B-BRAND] клубникаформа[B-TYPE] для[O] выпечки[O]', ['B-TYPE', 'B-BRAND', 'B-TYPE', 'O', 'O'])

Input: фарш свиной
Output: ('фарш[B-TYPE] свиной[I-TYPE]', ['B-TYPE', 'I-TYPE'])

Input: сок ананасовый без сахара
Output: ('сок[B-TYPE] ананасовый[I-TYPE] без[O] сахара[O]', ['B-TYPE', 'I-TYPE', 'O', 'O'])

Input: еринги
Output: ('еринги[B-TYPE]', ['B-TYPE'])

Input: молооко
Output: ('молооко[B-TYPE]

In [25]:
# from google.colab import drive
# drive.mount('/content/drive')

In [26]:
!zip -r best_ner_model1.zip /content/best_ner_model

"zip" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [27]:
# Ячейка для генерации submission.csv на основе файла submission-ner-5.csv
import pandas as pd
from tqdm.auto import tqdm

sub_path = "https://gist.githubusercontent.com/Niptlox/77f2302eb799de55081723f63555b524/raw/6602b39c929ff4a6308f051b93bf48fb2559733a/submission-ner-5.csv"
# sub_path = "synthetic_products_dataset_150mb.csv"
# Чтение файла с sample'ами
submission_df = pd.read_csv(
    sub_path,
    sep=";"
)

# Функция, которая преобразует вывод predict в список аннотаций [(start, end, tag), ...]
def format_annotation(text, tagged_output):
    tagtext, tags = tagged_output
    ann = []
    idx = 0
    for token, tag in zip(text.split(), tags):
        length = len(token)
        if tag:
            ann.append((idx, idx + length, tag))
        idx += length + 1  # +1 на разделитель пробела
    return ann

# Обработка всех sample и запись аннотаций
tqdm.pandas()
submission_df["annotation"] = submission_df["sample"].progress_apply(
    lambda s: format_annotation(s, predict(s))
)

ts = tuple(pd.Timestamp.now().timetuple()[2:6])
# Сохранение в CSV
submission_df.to_csv(f"submission-rbt-{ts}.csv", index=False, sep=";")

print("Файл успешно сгенерирован!", f"submission-rbt-{ts}.csv")

  0%|          | 0/5000 [00:00<?, ?it/s]

Файл успешно сгенерирован! submission-rbt-(2, 4, 56, 28).csv



```markdown
Проект Colab готов к запуску: весь код без сокращений, от загрузки данных до тренировки и тестирования модели.
```

<span style="display:none">[^1]</span>

<div align="center">⁂</div>

[^1]: https://gist.githubusercontent.com/Niptlox/77f2302eb799de55081723f63555b524/raw/82522906d4922f1974c339e7f74552a6219b3bb9/train-ner-5.csv

